# Using Reddit's API for Predicting Comments

In this project, we will practice two major skills. Collecting data via an API request and then building a binary predictor.

As we discussed in week 2, and earlier today, there are two components to starting a data science problem: the problem statement, and acquiring the data.

For this article, your problem statement will be: _What characteristics of a post on Reddit contribute most to the overall interaction (as measured by number of comments)?_

Your method for acquiring the data will be scraping the 'hot' threads as listed on the [Reddit homepage](https://www.reddit.com/). You'll acquire _AT LEAST FOUR_ pieces of information about each thread:
1. The title of the thread
2. The subreddit that the thread corresponds to
3. The length of time it has been up on Reddit
4. The number of comments on the thread

Once you've got the data, you will build a classification model that, using Natural Language Processing and any other relevant features, predicts whether or not a given Reddit post will have above or below the _median_ number of comments.

**BONUS PROBLEMS**
1. If creating a logistic regression, GridSearch Ridge and Lasso for this model and report the best hyperparameter values.
1. Scrape the actual text of the threads using Selenium (you'll learn about this in Webscraping II).
2. Write the actual article that you're pitching and turn it into a blog post that you host on your personal website.

### Scraping Thread Info from Reddit.com

#### Set up a request (using requests) to the URL below. 

*NOTE*: Reddit will throw a [429 error](https://httpstatuses.com/429) when using the following code:
```python
res = requests.get(URL)
```

This is because Reddit has throttled python's default user agent. You'll need to set a custom `User-agent` to get your request to work.
```python
res = requests.get(URL, headers={'User-agent': 'YOUR NAME Bot 0.1'})
```

In [8]:
import pandas as pd
import requests
import json

url_base = "http://www.reddit.com/"

slug_hot = "hot.json"

slug_subreddit = "r/boston/" # an optional intermediate slug to throw in to view a specific subreddit's 'hot' page

user = {'User-agent': 'Jon Withers Bot 0.2'} # I need a User-agent to get in

res = requests.get(url_base + slug_hot, headers = user) # put slug_subreddit in if you want

res.status_code

200

This is the expected number of children: there should be 25 posts on Reddit's front page.

Each of the 25 `children` have these attributes.

#### Use `res.json()` to convert the response into a dictionary format and set this to a variable. 

```python
data = res.json()
```

In [11]:
data = res.json()

# all_keys = data['data']['children'][0]['data'].keys()
# all_keys -= ['preview', 'post_hint']
# things_to_get = list(all_keys)

things_to_get = ['title','subreddit','created_utc','num_comments', 'score']

list_of_dicts = []
for i in range(25):
    this_dict = {}
    for j in range(len(things_to_get)):

        this_key = things_to_get[j]
        this_dict[this_key] = data['data']['children'][i]['data'][this_key]
    list_of_dicts.append(this_dict)

In [12]:
pd.DataFrame(list_of_dicts)

,created_utc,num_comments,score,subreddit,title
0,1.527616e+09,5167,15689,television,‘Roseanne’ Cancelled: ABC Scraps Season 11 Aft...
1,1.527616e+09,3252,13337,news,ABC cancels Roseanne Barr's sitcom after her t...
2,1.527609e+09,1470,32992,creepy,A friend went for a walk the other night and s...
3,1.527614e+09,510,9880,mildlyinfuriating,North Face stole my photo and put it on their ...
4,1.527608e+09,922,43964,BlackPeopleTwitter,Advanced Pettiness™️
5,1.527607e+09,1894,19651,videos,Millionaire televangelists Kenneth Copeland an...
6,1.527606e+09,712,22215,nottheonion,Please Don't Roast Marshmallows Over the Erupt...
7,1.527607e+09,415,16015,funny,Apologies Accepted.
8,1.527604e+09,1297,43171,todayilearned,TIL cartographers protect their intellectual p...
9,1.527605e+09,240,17602,DunderMifflin,"Since its lemonade stand season, lets apprecia..."


#### Getting more results

By default, Reddit will give you the top 25 posts:

```python
print(len(data['data']['children']))
```

If you want more, you'll need to do two things:
1. Get the name of the last post: `data['data']['after']`
2. Use that name to hit the following url: `http://www.reddit.com/hot.json?after=THE_AFTER_FROM_STEP_1`
3. Create a loop to repeat steps 1 and 2 until you have a sufficient number of posts. 

*NOTE*: Reddit will limit the number of requests per second you're allowed to make. When you create your loop, be sure to add the following after each iteration.

```python
time.sleep(3) # sleeps 3 seconds before continuing```

This will throttle your loop and keep you within Reddit's guidelines. You'll need to import the `time` library for this to work!

In [95]:
list_of_dicts = []
import time
time.time()

1527278136.044412

In [97]:
current_time = time.time()
for k in range(40):
    time.sleep(3)
    if k == 0:
        res = requests.get(url_base + slug_hot, user)
    else:
        res = requests.get(url_base + slug_hot + '?after=' + aft, user)
    
    things_to_get = list(all_keys)

    for i in range(25):
        this_dict = {}
        for j in range(len(things_to_get)):

            this_key = things_to_get[j]
            this_dict[this_key] = data['data']['children'][i]['data'][this_key]
        list_of_dicts.append(this_dict)
    aft = data['data']['after']

In [166]:
df = pd.DataFrame(list_of_dicts)

In [167]:
df['time_since_creation'] = current_time - df['created_utc']

In [168]:
len(df.columns)

79

In [170]:
df1 = df.dropna(how='all', axis=1)

In [171]:
len(df1.columns)

62

In [172]:
df2 = df.dropna(thresh=(0.5* 1025), axis = 1)
len(df.dropna(thresh=(0.5 * 1025), axis = 1).columns)

56

In [173]:
sorted(df2.isna().sum()[df.isna().sum()!=0])

[123, 123]

In [165]:
df2.to_csv('scraped_friday.csv')

In [184]:
df.loc[7,'title']

'Texas Republican Who Pushed To Impeach Obama Just Got Jailed After Being Convicted Of 23 Felonies'

## (Optional) Collect more information

While we only require you to collect four features, there may be other info that you can find on the results page that might be useful. Feel free to write more functions so that you have more interesting and useful data.

In [ ]:
## YOUR CODE HERE

### Save your results as a CSV
You may do this regularly while scraping data as well, so that if your scraper stops of your computer crashes, you don't lose all your data.

In [ ]:
# Export to csv


## Predicting comments using Random Forests + Another Classifier

#### Load in the the data of scraped results

In [ ]:
## YOUR CODE HERE

#### We want to predict a binary variable - whether the number of comments was low or high. Compute the median number of comments and create a new binary variable that is true when the number of comments is high (above the median)

We could also perform Linear Regression (or any regression) to predict the number of comments here. Instead, we are going to convert this into a _binary_ classification problem, by predicting two classes, HIGH vs LOW number of comments.

While performing regression may be better, performing classification may help remove some of the noise of the extremely popular threads. We don't _have_ to choose the `median` as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of comment numbers. 

In [ ]:
## YOUR CODE HERE

#### Thought experiment: What is the baseline accuracy for this model?

In [ ]:
## YOUR CODE HERE

#### Create a Random Forest model to predict High/Low number of comments using Sklearn. Start by ONLY using the subreddit as a feature. 

In [ ]:
## YOUR CODE HERE

#### Create a few new variables in your dataframe to represent interesting features of a thread title.
- For example, create a feature that represents whether 'cat' is in the title or whether 'funny' is in the title. 
- Then build a new Random Forest with these features. Do they add any value?
- After creating these variables, use count-vectorizer to create features based on the words in the thread titles.
- Build a new random forest model with subreddit and these new features included.

In [ ]:
## YOUR CODE HERE

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy of the model, as well as any other metrics you feel are appropriate. 

In [ ]:
## YOUR CODE HERE

#### Repeat the model-building process with a non-tree-based method.

In [ ]:
## YOUR CODE HERE

#### Use Count Vectorizer from scikit-learn to create features from the thread titles. 
- Examine using count or binary features in the model
- Re-evaluate your models using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [ ]:
## YOUR CODE HERE

# Executive Summary
---
Put your executive summary in a Markdown cell below.

### BONUS
Refer to the README for the bonus parts

In [ ]:
## YOUR CODE HERE